In [2]:


# Directory containing the files
dir_path = '229r/all1'

# Directory containing the files
dat_dir_path = 'data/229r/all1'


In [3]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
file_name = """229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt"""

def plot_graph(file_name="""229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt""", title='Average Fitness over Time', width = 800,    height = 600, log_y=True):

    # Read data from file into a list of strings
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Parse the lines to create a list of dictionaries, where each dictionary contains values for each column
    data = []
    for line in lines:
        if line.startswith('UD:'):
            values = line.strip().split()
            # print(values)

            data.append({
                'UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
            })

    # Create dataframe from list of dictionaries
    df = pd.DataFrame(data)
        

    # Define x and y data
    x_data = df.iloc[:, 0] # First column as x-axis
    y_data = df[["Fit"]]
    # y_data = df.iloc[:, 1:] # Every other column as y-axis

    # Create plotly trace for each column
    data = []
    for column in y_data.columns:
        trace = go.Scatter(x=x_data, y=y_data[column], mode='lines', name=column)
        data.append(trace)

    # Define plot layout
    layout = go.Layout(title=title, xaxis=dict(title='# of updates'), yaxis=dict(title='average fitness'), margin=dict(t=40, b=20, l=20, r=20), autosize=False, width=width, height=height, yaxis_type='log' if log_y else 'linear')

    # Create plotly figure
    fig = go.Figure(data=data, layout=layout)

    # Display plot
    file_parts = file_name.split('/')
    fig.show()
    pio.write_image(fig, file_parts[0]+'/plots/'+file_parts[1].split('.')[0]+'.png', width=width, height=height, scale=2)


In [4]:
import os
import pandas as pd
import re
# List to store the file names and last lines
file_data = []

# Regular expression pattern
pattern = r"(\d+)_mxc_([\w-]+)_(\[.*\])_date_(\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_xydict_values\((\[.*\])\)\.txt"
# Create an empty DataFrame with the required columns
column_names = ['index', 'max_count', 'values', 'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs']
df = pd.DataFrame(columns=column_names)

# Create an empty list to store the data
data_list = []

# Loop through all the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file and read the last line
        with open(os.path.join(dir_path, filename), 'r') as f:
            # Search for pattern in the log_file_name
            match = re.search(pattern, filename)
            if not match:
                raise Exception("No match found for pattern in file name: {}".format(filename))
            index = int(match.group(1))
            max_count = match.group(2)
            if max_count == 'False':
                max_count = -1
            elif max_count == 'True':
                max_count = 0
            else:
                max_count = int(match.group(2))
                # continue
            
            values = tuple(int(i) for i in match.group(3).strip('[]').split(', '))
            experiment_start_time_string = match.group(4) + "_" + match.group(5)
            xy_values = tuple(int(i) for i in match.group(6).strip('[]').split(', '))
            
            lines = f.readlines()
            last_line = lines[-1].strip()
            
            with open(os.path.join(dat_dir_path, filename), 'r') as fdat: 
                lines_dat = fdat.readlines()
                last_line_dat = lines_dat[-1].strip()
                numbers = last_line_dat.split()

                dictionary = {
                    "Update": int(numbers[0]),
                    "Not": int(numbers[1]),
                    "Nand": int(numbers[2]),
                    "And": int(numbers[3]),
                    "OrNot": int(numbers[4]),
                    "Or": int(numbers[5]),
                    "AndNot": int(numbers[6]),
                    "Nor": int(numbers[7]),
                    "Xor": int(numbers[8])
                }
                EQU_dict = {
                    "Equals": int(numbers[9])}

            
            if last_line.startswith('UD:'):
                last_line_string = last_line.strip().split()

                data = {
                'max_count': max_count,
                'values': values,
                'last_line_string': last_line_string,
                'experiment_start_time_string': experiment_start_time_string,
                'xy_values': xy_values,
                'Last_UD': int(last_line_string[1]),
                'Gen': float(last_line_string[3]),
                'Fit': float(last_line_string[5]),
                'Orgs': int(last_line_string[7]),
                'index': index,
                'filename': filename,
                }
                data.update(dictionary)
                EQU_dict.update(data)

                # Append the data to the list
                data_list.append(EQU_dict)
            else: 
                print("Last line does not start with UD: {}".format(filename))
                continue 

# Create the DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the dataframe
df


Last line does not start with UD: 13_mxc_True_[1, 2, 4, 3, 5]_date_04-17_10-49-22_xydict_values([120, 120]).txt


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,0,"(2, 3, 1, 4, 5)","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"(120, 120)",4400,1017.6480,5.304728e+72,14345,...,"0_mxc_True_[2, 3, 1, 4, 5]_date_04-13_20-35-15...",4400,12397,13294,11875,13104,49,542,9074,0
1,2,1,"(2, 4, 3, 1, 5)","[UD:, 10000, Gen:, 3025.315, Fit:, 75508.13, O...",04-18_00-00-52,"(120, 120)",10000,3025.3150,7.550813e+04,14379,...,"14_mxc_1_[2, 4, 3, 1, 5]_date_04-18_00-00-52_x...",10000,13663,13027,12022,12789,12783,11770,7,0
2,0,-1,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 9906.699, Fit:, 1.068264e+9...",04-17_16-23-01,"(120, 120)",10000,9906.6990,1.068264e+90,11072,...,"14_mxc_-1_[1, 3, 2, 4, 5]_date_04-17_16-23-01_...",10000,2681,2130,546,4806,16,5,0,0
3,3389,0,"(4, 1, 2, 3, 5)","[UD:, 3100, Gen:, 792.0930, Fit:, 9.769858e+44...",04-15_03-32-04,"(120, 120)",3100,792.0930,9.769858e+44,14360,...,"3_mxc_True_[4, 1, 2, 3, 5]_date_04-15_03-32-04...",3100,12099,12937,315,11570,2853,10150,4093,7
4,9140,0,"(1, 4, 2, 3, 5)","[UD:, 6200, Gen:, 1636.221, Fit:, 1.244618e+22...",04-16_03-30-29,"(120, 120)",6200,1636.2210,1.244618e+220,14310,...,"8_mxc_True_[1, 4, 2, 3, 5]_date_04-16_03-30-29...",6200,12429,13096,12429,13473,11883,12242,407,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,0,-1,"(1, 2, 3, 4, 4)","[UD:, 10000, Gen:, 379.9630, Fit:, 5.925234e+1...",04-17_02-04-23,"(120, 120)",10000,379.9630,5.925234e+163,12514,...,"12_mxc_False_[1, 2, 3, 4, 4]_date_04-17_02-04-...",10000,696,890,467,3541,3621,640,1589,0
1470,0,0,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2267.469, Fit:, 4658.941, O...",04-22_01-04-11,"(120, 120)",10000,2267.4690,4.658941e+03,14386,...,"21_mxc_0_[1, 2, 3, 4, 1]_date_04-22_01-04-11_x...",10000,13584,13493,145,13380,13002,13042,13036,0
1471,0,-1,"(1, 2, 4, 3, 5)","[UD:, 9400, Gen:, 3559.430, Fit:, 5.733812e+25...",04-18_10-25-05,"(120, 120)",9400,3559.4300,5.733812e+254,14199,...,"15_mxc_-1_[1, 2, 4, 3, 5]_date_04-18_10-25-05_...",9400,399,11795,151,174,7093,11153,0,3
1472,0,0,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2650.036, Fit:, 26249.29, O...",04-18_03-42-57,"(120, 120)",10000,2650.0360,2.624929e+04,14367,...,"14_mxc_0_[4, 1, 2, 3, 5]_date_04-18_03-42-57_x...",10000,13013,11787,8818,12283,12266,11097,12046,0


In [5]:
import pandas as pd
import pickle

# Pickle the dataframe to a file
with open('229r/results/all1_rp3_fitness_and_counts_df.pickle', 'wb') as f:
    pickle.dump(df, f)


In [6]:
import pandas as pd
import pickle

# Load the pickled dataframe from the file
with open('229r/results/all1_rp3_fitness_and_counts_df.pickle', 'rb') as f:
    df = pickle.load(f)

# Print the dataframe
display(df)


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,0,"(2, 3, 1, 4, 5)","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"(120, 120)",4400,1017.6480,5.304728e+72,14345,...,"0_mxc_True_[2, 3, 1, 4, 5]_date_04-13_20-35-15...",4400,12397,13294,11875,13104,49,542,9074,0
1,2,1,"(2, 4, 3, 1, 5)","[UD:, 10000, Gen:, 3025.315, Fit:, 75508.13, O...",04-18_00-00-52,"(120, 120)",10000,3025.3150,7.550813e+04,14379,...,"14_mxc_1_[2, 4, 3, 1, 5]_date_04-18_00-00-52_x...",10000,13663,13027,12022,12789,12783,11770,7,0
2,0,-1,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 9906.699, Fit:, 1.068264e+9...",04-17_16-23-01,"(120, 120)",10000,9906.6990,1.068264e+90,11072,...,"14_mxc_-1_[1, 3, 2, 4, 5]_date_04-17_16-23-01_...",10000,2681,2130,546,4806,16,5,0,0
3,3389,0,"(4, 1, 2, 3, 5)","[UD:, 3100, Gen:, 792.0930, Fit:, 9.769858e+44...",04-15_03-32-04,"(120, 120)",3100,792.0930,9.769858e+44,14360,...,"3_mxc_True_[4, 1, 2, 3, 5]_date_04-15_03-32-04...",3100,12099,12937,315,11570,2853,10150,4093,7
4,9140,0,"(1, 4, 2, 3, 5)","[UD:, 6200, Gen:, 1636.221, Fit:, 1.244618e+22...",04-16_03-30-29,"(120, 120)",6200,1636.2210,1.244618e+220,14310,...,"8_mxc_True_[1, 4, 2, 3, 5]_date_04-16_03-30-29...",6200,12429,13096,12429,13473,11883,12242,407,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,0,-1,"(1, 2, 3, 4, 4)","[UD:, 10000, Gen:, 379.9630, Fit:, 5.925234e+1...",04-17_02-04-23,"(120, 120)",10000,379.9630,5.925234e+163,12514,...,"12_mxc_False_[1, 2, 3, 4, 4]_date_04-17_02-04-...",10000,696,890,467,3541,3621,640,1589,0
1470,0,0,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2267.469, Fit:, 4658.941, O...",04-22_01-04-11,"(120, 120)",10000,2267.4690,4.658941e+03,14386,...,"21_mxc_0_[1, 2, 3, 4, 1]_date_04-22_01-04-11_x...",10000,13584,13493,145,13380,13002,13042,13036,0
1471,0,-1,"(1, 2, 4, 3, 5)","[UD:, 9400, Gen:, 3559.430, Fit:, 5.733812e+25...",04-18_10-25-05,"(120, 120)",9400,3559.4300,5.733812e+254,14199,...,"15_mxc_-1_[1, 2, 4, 3, 5]_date_04-18_10-25-05_...",9400,399,11795,151,174,7093,11153,0,3
1472,0,0,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2650.036, Fit:, 26249.29, O...",04-18_03-42-57,"(120, 120)",10000,2650.0360,2.624929e+04,14367,...,"14_mxc_0_[4, 1, 2, 3, 5]_date_04-18_03-42-57_x...",10000,13013,11787,8818,12283,12266,11097,12046,0


In [7]:
df.describe()

,Equals,max_count,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
count,1474.000000,1474.000000,1474.000000,1474.000000,1.474000e+03,1474.00000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000,1474.000000
mean,2480.545455,-0.199457,7225.283582,1782.403663,1.024477e+298,13280.76730,13.983718,7224.898236,9322.046811,9027.867028,4944.209634,8809.550204,6472.417910,6654.879240,5010.907056,1033.529851
std,4164.771254,0.746733,3661.041560,1287.324373,inf,2191.07722,7.767917,3661.429597,4897.565581,5100.204814,5563.291859,5066.926424,5522.346182,5712.628087,5413.803579,2985.921577
min,0.000000,-1.000000,500.000000,45.979350,2.501915e-01,3370.00000,0.000000,500.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-1.000000,3100.000000,633.130075,3.178987e+04,13734.25000,8.000000,3100.000000,4505.250000,3946.000000,12.000000,3496.750000,29.000000,43.500000,1.250000,0.000000
50%,0.000000,0.000000,10000.000000,1960.112500,1.123414e+08,14322.00000,15.000000,10000.000000,12413.000000,12148.500000,652.000000,11898.000000,8359.500000,9249.000000,1247.500000,0.000000
75%,4049.750000,0.000000,10000.000000,2421.019000,7.803228e+132,14367.00000,21.000000,10000.000000,13229.750000,13262.750000,11533.750000,12893.750000,11914.500000,12230.000000,11345.000000,2.000000
max,13372.000000,1.000000,10000.000000,13914.040000,5.375361e+300,14392.00000,25.000000,10000.000000,14061.000000,14174.000000,13452.000000,14077.000000,13722.000000,13855.000000,13380.000000,12445.000000


In [8]:
df.columns

Index(['Equals', 'max_count', 'values', 'last_line_string',
       'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit',
       'Orgs', 'index', 'filename', 'Update', 'Not', 'Nand', 'And', 'OrNot',
       'Or', 'AndNot', 'Nor', 'Xor'],
      dtype='object')

In [9]:
# Get all the rows with max_count = False and Last_UD = 10000
filtered_df = df[(df['max_count'] == False) & (df['Last_UD'] == 10000)]

In [10]:
filtered_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
5,0,0,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 2466.994, Fit:, 16702.16, O...",04-16_23-19-10,"(120, 120)",10000,2466.994,1.670216e+04,14378,...,"11_mxc_True_[3, 2, 1, 4, 5]_date_04-16_23-19-1...",10000,13563,13626,12277,13049,12517,12754,12807,3
6,3641,0,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 2538.028, Fit:, 4.369271e+1...",04-15_05-00-46,"(120, 120)",10000,2538.028,4.369271e+110,14248,...,"4_mxc_True_[1, 2, 3, 4, 6]_date_04-15_05-00-46...",10000,12988,10536,7502,12010,12113,12095,11874,8
7,0,0,"(2, 4, 3, 1, 5)","[UD:, 10000, Gen:, 2327.617, Fit:, 108411.5, O...",04-16_14-03-18,"(120, 120)",10000,2327.617,1.084115e+05,14362,...,"9_mxc_True_[2, 4, 3, 1, 5]_date_04-16_14-03-18...",10000,13598,13432,12474,12575,12249,12949,9098,0
12,0,0,"(1, 4, 2, 3, 5)","[UD:, 10000, Gen:, 2139.232, Fit:, 6436.750, O...",04-19_01-01-47,"(120, 120)",10000,2139.232,6.436750e+03,14381,...,"16_mxc_0_[1, 4, 2, 3, 5]_date_04-19_01-01-47_x...",10000,13292,13071,11925,12860,12380,4427,8338,0
16,0,0,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 1971.971, Fit:, 8111.602, O...",04-20_20-27-19,"(120, 120)",10000,1971.971,8.111602e+03,14372,...,"19_mxc_0_[2, 4, 1, 3, 5]_date_04-20_20-27-19_x...",10000,13089,12615,13266,12265,32,157,11552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,0,0,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 2374.591, Fit:, 66773.51, O...",04-16_17-34-59,"(120, 120)",10000,2374.591,6.677351e+04,14369,...,"9_mxc_True_[4, 3, 2, 1, 5]_date_04-16_17-34-59...",10000,13459,12430,11210,11714,10932,13153,28,0
1464,3512,0,"(2, 3, 1, 4, 5)","[UD:, 10000, Gen:, 1133.529, Fit:, 4.690366e+1...",04-21_11-55-14,"(120, 120)",10000,1133.529,4.690366e+19,14373,...,"20_mxc_0_[2, 3, 1, 4, 5]_date_04-21_11-55-14_x...",10000,11353,12723,4559,12986,4171,209,4151,0
1467,0,0,"(3, 1, 4, 2, 5)","[UD:, 10000, Gen:, 2230.110, Fit:, 39000.65, O...",04-14_13-33-19,"(120, 120)",10000,2230.110,3.900065e+04,14347,...,"2_mxc_True_[3, 1, 4, 2, 5]_date_04-14_13-33-19...",10000,13569,12583,10077,11806,13063,11254,12363,0
1470,0,0,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2267.469, Fit:, 4658.941, O...",04-22_01-04-11,"(120, 120)",10000,2267.469,4.658941e+03,14386,...,"21_mxc_0_[1, 2, 3, 4, 1]_date_04-22_01-04-11_x...",10000,13584,13493,145,13380,13002,13042,13036,0


In [11]:
sorted_EQU_df = df.sort_values("Equals", ascending=False)

In [12]:
sorted_EQU_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
1310,13372,0,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3333.767, Fit:, 9.435327e+0...",04-17_16-05-58,"(120, 120)",10000,3333.7670,9.435327e+09,14234,...,"14_mxc_0_[1, 3, 2, 4, 5]_date_04-17_16-05-58_x...",10000,14009,13303,12028,13068,12673,13388,13349,16
1440,12723,0,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2169.567, Fit:, 5.271931e+2...",04-14_15-15-07,"(120, 120)",10000,2169.5670,5.271931e+27,14260,...,"2_mxc_True_[4, 1, 2, 3, 5]_date_04-14_15-15-07...",10000,13532,13302,649,13460,13100,5681,11592,12445
298,12262,0,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 3144.089, Fit:, 2.497056e+1...",04-23_09-52-11,"(120, 120)",10000,3144.0890,2.497056e+18,14134,...,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x...",10000,13364,13083,12682,13388,9921,12503,850,1
264,12187,1,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 2762.624, Fit:, 124135.0, O...",04-25_02-22-08,"(120, 120)",10000,2762.6240,1.241350e+05,14369,...,"25_mxc_1_[1, 3, 2, 4, 5]_date_04-25_02-22-08_x...",10000,12914,13143,12848,13082,12850,656,12517,1
743,12167,1,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 2783.730, Fit:, 2285600., O...",04-20_08-44-03,"(120, 120)",10000,2783.7300,2.285600e+06,14361,...,"18_mxc_1_[4, 3, 2, 1, 5]_date_04-20_08-44-03_x...",10000,13511,12969,12071,13206,13103,12795,2379,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,0,"(3, 4, 1, 2, 5)","[UD:, 10000, Gen:, 2303.425, Fit:, 17465.97, O...",04-17_07-14-46,"(120, 120)",10000,2303.4250,1.746597e+04,14385,...,"12_mxc_True_[3, 4, 1, 2, 5]_date_04-17_07-14-4...",10000,13346,13268,12277,13410,12148,11607,1,0
810,0,1,"(1, 2, 3, 4, 4)","[UD:, 10000, Gen:, 2656.523, Fit:, 13392.74, O...",04-24_22-48-42,"(120, 120)",10000,2656.5230,1.339274e+04,14355,...,"25_mxc_1_[1, 2, 3, 4, 4]_date_04-24_22-48-42_x...",10000,13107,13144,11786,11820,11390,13016,11470,0
249,0,1,"(1, 2, 3, 4, 3)","[UD:, 10000, Gen:, 2039.397, Fit:, 7632.924, O...",04-18_19-45-38,"(120, 120)",10000,2039.3970,7.632924e+03,14334,...,"16_mxc_1_[1, 2, 3, 4, 3]_date_04-18_19-45-38_x...",10000,11183,11167,10122,10732,10409,12291,10426,2
247,0,1,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2036.659, Fit:, 4523.619, O...",04-22_17-01-25,"(120, 120)",10000,2036.6590,4.523619e+03,14389,...,"22_mxc_1_[1, 2, 3, 4, 1]_date_04-22_17-01-25_x...",10000,13682,13318,103,12850,11421,11947,12719,0


In [13]:
cleaned_df = sorted_EQU_df[sorted_EQU_df['Last_UD'] == 10000]
display(cleaned_df)

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
1310,13372,0,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3333.767, Fit:, 9.435327e+0...",04-17_16-05-58,"(120, 120)",10000,3333.767,9.435327e+09,14234,...,"14_mxc_0_[1, 3, 2, 4, 5]_date_04-17_16-05-58_x...",10000,14009,13303,12028,13068,12673,13388,13349,16
1440,12723,0,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2169.567, Fit:, 5.271931e+2...",04-14_15-15-07,"(120, 120)",10000,2169.567,5.271931e+27,14260,...,"2_mxc_True_[4, 1, 2, 3, 5]_date_04-14_15-15-07...",10000,13532,13302,649,13460,13100,5681,11592,12445
298,12262,0,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 3144.089, Fit:, 2.497056e+1...",04-23_09-52-11,"(120, 120)",10000,3144.089,2.497056e+18,14134,...,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x...",10000,13364,13083,12682,13388,9921,12503,850,1
264,12187,1,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 2762.624, Fit:, 124135.0, O...",04-25_02-22-08,"(120, 120)",10000,2762.624,1.241350e+05,14369,...,"25_mxc_1_[1, 3, 2, 4, 5]_date_04-25_02-22-08_x...",10000,12914,13143,12848,13082,12850,656,12517,1
743,12167,1,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 2783.730, Fit:, 2285600., O...",04-20_08-44-03,"(120, 120)",10000,2783.730,2.285600e+06,14361,...,"18_mxc_1_[4, 3, 2, 1, 5]_date_04-20_08-44-03_x...",10000,13511,12969,12071,13206,13103,12795,2379,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,0,0,"(4, 2, 1, 3, 5)","[UD:, 10000, Gen:, 1740.318, Fit:, 20971.63, O...",04-15_16-42-37,"(120, 120)",10000,1740.318,2.097163e+04,14362,...,"5_mxc_True_[4, 2, 1, 3, 5]_date_04-15_16-42-37...",10000,12886,13753,12154,13208,11850,10836,10961,2
250,0,0,"(3, 4, 1, 2, 5)","[UD:, 10000, Gen:, 2303.425, Fit:, 17465.97, O...",04-17_07-14-46,"(120, 120)",10000,2303.425,1.746597e+04,14385,...,"12_mxc_True_[3, 4, 1, 2, 5]_date_04-17_07-14-4...",10000,13346,13268,12277,13410,12148,11607,1,0
810,0,1,"(1, 2, 3, 4, 4)","[UD:, 10000, Gen:, 2656.523, Fit:, 13392.74, O...",04-24_22-48-42,"(120, 120)",10000,2656.523,1.339274e+04,14355,...,"25_mxc_1_[1, 2, 3, 4, 4]_date_04-24_22-48-42_x...",10000,13107,13144,11786,11820,11390,13016,11470,0
249,0,1,"(1, 2, 3, 4, 3)","[UD:, 10000, Gen:, 2039.397, Fit:, 7632.924, O...",04-18_19-45-38,"(120, 120)",10000,2039.397,7.632924e+03,14334,...,"16_mxc_1_[1, 2, 3, 4, 3]_date_04-18_19-45-38_x...",10000,11183,11167,10122,10732,10409,12291,10426,2


In [14]:
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"] == 0]
d1 = d1[["Equals", "Fit"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &        Equals &            Fit \\
\midrule
mean &   2755.153846 &  9.253921e+295 \\
min  &      0.000000 &   6.751146e+01 \\
25\%  &      0.000000 &   1.445213e+04 \\
50\%  &      0.000000 &   4.964901e+04 \\
75\%  &   6435.000000 &   1.252362e+07 \\
max  &  13372.000000 &  3.969932e+298 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_45608/426925932.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [15]:
# display(cleaned_df[~cleaned_df["max_count"]].describe())
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"] == -1]
d1 = d1[["Equals", "Fit", "filename"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &       Equals &            Fit \\
\midrule
mean &    23.130769 &  4.504708e+292 \\
min  &     0.000000 &   7.534021e+03 \\
25\%  &     0.000000 &   2.031529e+58 \\
50\%  &     0.000000 &  5.661669e+102 \\
75\%  &     0.000000 &  4.443928e+163 \\
max  &  2799.000000 &  3.974209e+294 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_45608/980696805.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [16]:
# display(cleaned_df[~cleaned_df["max_count"]].describe())
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"] == 1]
d1 = d1[["Equals", "Fit", "filename"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &        Equals &           Fit \\
\midrule
mean &   4965.955326 &  7.864197e+05 \\
min  &      0.000000 &  4.085866e+02 \\
25\%  &      0.000000 &  1.611197e+04 \\
50\%  &     43.000000 &  7.550813e+04 \\
75\%  &  10634.500000 &  8.140241e+05 \\
max  &  12187.000000 &  9.594949e+06 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_45608/2692458926.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [17]:
cleaned_df = cleaned_df[["values", "max_count", "Equals", "Fit", "index", "filename"]]

In [18]:
diff_vals_df = cleaned_df.sort_values("values")

In [19]:
print(diff_vals_df.to_latex(index=False))

/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_45608/128294827.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(diff_vals_df.to_latex(index=False))


\begin{tabular}{lrrrrl}
\toprule
         values &  max\_count &  Equals &           Fit &  index &                                           filename \\
\midrule
(1, 2, 3, 4, 1) &          1 &       0 &  4.523619e+03 &     22 & 22\_mxc\_1\_[1, 2, 3, 4, 1]\_date\_04-22\_17-01-25\_xy... \\
(1, 2, 3, 4, 1) &         -1 &       0 & 8.290687e+108 &     16 & 16\_mxc\_-1\_[1, 2, 3, 4, 1]\_date\_04-18\_21-36-55\_x... \\
(1, 2, 3, 4, 1) &          0 &       0 &  1.650670e+04 &     14 & 14\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-17\_14-09-43\_xy... \\
(1, 2, 3, 4, 1) &          0 &       0 &  1.468369e+04 &     16 & 16\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-18\_21-21-11\_xy... \\
(1, 2, 3, 4, 1) &         -1 &       0 &  9.591164e+12 &      5 & 5\_mxc\_False\_[1, 2, 3, 4, 1]\_date\_04-15\_10-56-28... \\
(1, 2, 3, 4, 1) &          1 &       0 &  1.548548e+04 &     23 & 23\_mxc\_1\_[1, 2, 3, 4, 1]\_date\_04-23\_12-22-14\_xy... \\
(1, 2, 3, 4, 1) &          0 &       0 &  1.741281e+04 &      8 & 8\_mxc\_Tr

In [20]:
display(diff_vals_df)

,values,max_count,Equals,Fit,index,filename
247,"(1, 2, 3, 4, 1)",1,0,4.523619e+03,22,"22_mxc_1_[1, 2, 3, 4, 1]_date_04-22_17-01-25_x..."
201,"(1, 2, 3, 4, 1)",-1,0,8.290687e+108,16,"16_mxc_-1_[1, 2, 3, 4, 1]_date_04-18_21-36-55_..."
877,"(1, 2, 3, 4, 1)",0,0,1.650670e+04,14,"14_mxc_0_[1, 2, 3, 4, 1]_date_04-17_14-09-43_x..."
1188,"(1, 2, 3, 4, 1)",0,0,1.468369e+04,16,"16_mxc_0_[1, 2, 3, 4, 1]_date_04-18_21-21-11_x..."
1185,"(1, 2, 3, 4, 1)",-1,0,9.591164e+12,5,"5_mxc_False_[1, 2, 3, 4, 1]_date_04-15_10-56-2..."
...,...,...,...,...,...,...
434,"(4, 3, 2, 1, 5)",1,0,1.076603e+05,20,"20_mxc_1_[4, 3, 2, 1, 5]_date_04-21_21-51-23_x..."
1292,"(4, 3, 2, 1, 5)",1,10591,1.367154e+06,16,"16_mxc_1_[4, 3, 2, 1, 5]_date_04-19_12-20-25_x..."
460,"(4, 3, 2, 1, 5)",1,0,6.449041e+04,21,"21_mxc_1_[4, 3, 2, 1, 5]_date_04-22_14-24-17_x..."
563,"(4, 3, 2, 1, 5)",0,0,7.765631e+04,18,"18_mxc_0_[4, 3, 2, 1, 5]_date_04-20_08-59-32_x..."


In [21]:
fdf = diff_vals_df 

In [22]:
fdf

,values,max_count,Equals,Fit,index,filename
247,"(1, 2, 3, 4, 1)",1,0,4.523619e+03,22,"22_mxc_1_[1, 2, 3, 4, 1]_date_04-22_17-01-25_x..."
201,"(1, 2, 3, 4, 1)",-1,0,8.290687e+108,16,"16_mxc_-1_[1, 2, 3, 4, 1]_date_04-18_21-36-55_..."
877,"(1, 2, 3, 4, 1)",0,0,1.650670e+04,14,"14_mxc_0_[1, 2, 3, 4, 1]_date_04-17_14-09-43_x..."
1188,"(1, 2, 3, 4, 1)",0,0,1.468369e+04,16,"16_mxc_0_[1, 2, 3, 4, 1]_date_04-18_21-21-11_x..."
1185,"(1, 2, 3, 4, 1)",-1,0,9.591164e+12,5,"5_mxc_False_[1, 2, 3, 4, 1]_date_04-15_10-56-2..."
...,...,...,...,...,...,...
434,"(4, 3, 2, 1, 5)",1,0,1.076603e+05,20,"20_mxc_1_[4, 3, 2, 1, 5]_date_04-21_21-51-23_x..."
1292,"(4, 3, 2, 1, 5)",1,10591,1.367154e+06,16,"16_mxc_1_[4, 3, 2, 1, 5]_date_04-19_12-20-25_x..."
460,"(4, 3, 2, 1, 5)",1,0,6.449041e+04,21,"21_mxc_1_[4, 3, 2, 1, 5]_date_04-22_14-24-17_x..."
563,"(4, 3, 2, 1, 5)",0,0,7.765631e+04,18,"18_mxc_0_[4, 3, 2, 1, 5]_date_04-20_08-59-32_x..."


In [23]:
gdf = diff_vals_df.groupby(["values", "max_count"]).mean()
gdf

Equals            Fit      index
values          max_count                                       
(1, 2, 3, 4, 1) -1            0.166667  4.146335e+153  14.666667
                 0          167.956522   1.285871e+18  12.652174
                 1            2.636364   2.816939e+04  19.727273
(1, 2, 3, 4, 2) -1            0.000000  2.064292e+144  16.800000
                 0         1274.363636   4.270341e+71  12.136364
...                                ...            ...        ...
(4, 3, 1, 2, 5)  0         2637.000000  3.678695e+161  11.833333
                 1         3359.571429   4.154634e+05  18.571429
(4, 3, 2, 1, 5) -1            0.000000   2.368785e+07  20.000000
                 0         2286.692308  1.384264e+186  10.846154
                 1         6503.714286   1.668229e+06  18.571429

[87 rows x 3 columns]

In [24]:
gdf = gdf.reset_index().sort_values("Equals")

In [25]:
gdf[gdf["max_count"] == 0]

,values,max_count,Equals,Fit,index
1,"(1, 2, 3, 4, 1)",0,167.956522,1.285871e+18,12.652174
4,"(1, 2, 3, 4, 2)",0,1274.363636,4.270341e+71,12.136364
43,"(2, 3, 4, 1, 5)",0,1532.000000,5.015398e+115,12.666667
10,"(1, 2, 3, 4, 4)",0,1757.947368,9.860744e+36,12.526316
40,"(2, 3, 1, 4, 5)",0,1878.266667,3.976403e+54,13.933333
34,"(2, 1, 3, 4, 5)",0,2206.333333,1.533905e+111,11.750000
49,"(2, 4, 3, 1, 5)",0,2206.375000,1.699892e+252,11.500000
79,"(4, 2, 3, 1, 5)",0,2219.615385,4.244302e+265,11.153846
25,"(1, 3, 4, 2, 5)",0,2281.666667,1.727977e+36,11.066667
85,"(4, 3, 2, 1, 5)",0,2286.692308,1.384264e+186,10.846154


In [40]:
pl1 = gdf[gdf["max_count"] == 1].reset_index(drop=True)

In [42]:
print(pl1.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &           values &  max\_count &       Equals &           Fit &      index \\
\midrule
0  &  (1, 2, 3, 4, 1) &          1 &     2.636364 &  2.816939e+04 &  19.727273 \\
1  &  (2, 4, 1, 3, 5) &          1 &  1438.727273 &  1.204840e+05 &  19.727273 \\
2  &  (1, 2, 3, 4, 2) &          1 &  1574.363636 &  6.361376e+04 &  19.727273 \\
3  &  (2, 3, 4, 1, 5) &          1 &  2866.454545 &  5.035790e+05 &  19.727273 \\
4  &  (4, 2, 3, 1, 5) &          1 &  3001.714286 &  5.046762e+05 &  18.571429 \\
5  &  (1, 2, 3, 4, 3) &          1 &  3297.727273 &  1.549893e+05 &  19.727273 \\
6  &  (4, 3, 1, 2, 5) &          1 &  3359.571429 &  4.154634e+05 &  18.571429 \\
7  &  (3, 4, 1, 2, 5) &          1 &  3549.000000 &  5.746923e+05 &  20.200000 \\
8  &  (3, 2, 1, 4, 5) &          1 &  4251.400000 &  6.187622e+05 &  20.200000 \\
9  &  (4, 2, 1, 3, 5) &          1 &  4628.857143 &  5.403472e+05 &  18.571429 \\
10 &  (1, 2, 3, 4, 4) &          1 &  4848.727273 &  5.

/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_45608/454812560.py:1: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [27]:
gdf[gdf["max_count"] == -1]

,values,max_count,Equals,Fit,index
63,"(3, 4, 1, 2, 5)",-1,0.000000,1.186134e+47,11.000000
42,"(2, 3, 4, 1, 5)",-1,0.000000,4.842656e+40,9.000000
24,"(1, 3, 4, 2, 5)",-1,0.000000,9.822800e+267,13.200000
48,"(2, 4, 3, 1, 5)",-1,0.000000,3.380110e+293,15.250000
51,"(3, 1, 2, 4, 5)",-1,0.000000,2.659785e+266,13.500000
21,"(1, 3, 2, 4, 5)",-1,0.000000,4.065488e+274,18.500000
54,"(3, 1, 4, 2, 5)",-1,0.000000,3.853638e+116,14.333333
57,"(3, 2, 1, 4, 5)",-1,0.000000,1.162867e+166,4.750000
60,"(3, 2, 4, 1, 5)",-1,0.000000,1.394834e+119,7.250000
36,"(2, 1, 4, 3, 5)",-1,0.000000,2.937517e+44,12.750000


In [28]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 1))]

,values,max_count,Equals,Fit,index
2,"(1, 2, 3, 4, 1)",1,2.636364,2.816939e+04,19.727273
5,"(1, 2, 3, 4, 2)",1,1574.363636,6.361376e+04,19.727273
8,"(1, 2, 3, 4, 3)",1,3297.727273,1.549893e+05,19.727273
11,"(1, 2, 3, 4, 4)",1,4848.727273,5.833734e+05,19.727273
17,"(1, 2, 3, 4, 6)",1,5931.500000,1.288548e+06,19.500000
32,"(1, 4, 3, 2, 5)",1,6044.818182,1.345442e+06,19.727273
14,"(1, 2, 3, 4, 5)",1,6074.636364,1.832694e+06,19.727273
29,"(1, 4, 2, 3, 5)",1,6331.272727,1.214553e+06,19.727273
20,"(1, 2, 4, 3, 5)",1,6637.000000,1.482069e+06,19.727273
26,"(1, 3, 4, 2, 5)",1,6961.636364,1.829892e+06,19.727273


In [29]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 4))]

,values,max_count,Equals,Fit,index
80,"(4, 2, 3, 1, 5)",1,3001.714286,5.046762e+05,18.571429
83,"(4, 3, 1, 2, 5)",1,3359.571429,4.154634e+05,18.571429
77,"(4, 2, 1, 3, 5)",1,4628.857143,5.403472e+05,18.571429
71,"(4, 1, 2, 3, 5)",1,5683.375000,1.536059e+05,19.125000
74,"(4, 1, 3, 2, 5)",1,6166.750000,1.074022e+06,19.125000
86,"(4, 3, 2, 1, 5)",1,6503.714286,1.668229e+06,18.571429


In [30]:
fdf[fdf["values"] == (4, 1, 2, 3, 5)]

,values,max_count,Equals,Fit,index,filename
1472,"(4, 1, 2, 3, 5)",0,0,2.624929e+04,14,"14_mxc_0_[4, 1, 2, 3, 5]_date_04-18_03-42-57_x..."
863,"(4, 1, 2, 3, 5)",-1,0,1.307566e+126,11,"11_mxc_False_[4, 1, 2, 3, 5]_date_04-17_00-32-..."
116,"(4, 1, 2, 3, 5)",0,11356,7.820336e+173,8,"8_mxc_True_[4, 1, 2, 3, 5]_date_04-16_06-55-43..."
515,"(4, 1, 2, 3, 5)",1,62,1.553304e+04,18,"18_mxc_1_[4, 1, 2, 3, 5]_date_04-20_05-55-51_x..."
954,"(4, 1, 2, 3, 5)",1,11765,1.385306e+05,20,"20_mxc_1_[4, 1, 2, 3, 5]_date_04-21_17-11-15_x..."
958,"(4, 1, 2, 3, 5)",0,11230,1.299495e+05,19,"19_mxc_0_[4, 1, 2, 3, 5]_date_04-21_01-28-48_x..."
156,"(4, 1, 2, 3, 5)",1,0,2.244936e+03,21,"21_mxc_1_[4, 1, 2, 3, 5]_date_04-22_10-47-24_x..."
763,"(4, 1, 2, 3, 5)",1,0,3.782427e+03,22,"22_mxc_1_[4, 1, 2, 3, 5]_date_04-23_06-57-24_x..."
1100,"(4, 1, 2, 3, 5)",0,0,4.212910e+03,9,"9_mxc_True_[4, 1, 2, 3, 5]_date_04-16_16-06-50..."
602,"(4, 1, 2, 3, 5)",-1,0,3.594581e+49,5,"5_mxc_False_[4, 1, 2, 3, 5]_date_04-15_16-30-4..."


In [31]:
fname = fdf[fdf["values"] == (4, 1, 2, 3, 5)]["filename"].iloc[0]

In [32]:
plot_graph(file_name=dir_path + "/" + fname)

In [33]:
def plot_vals(vals, max_count, log_y = True, i = 0):
    fname = fdf[(fdf["values"] == vals) & (fdf["max_count"] == max_count)]["filename"].iloc[i]
    plot_graph(file_name=dir_path + "/" + fname, title=f"""Average Fitness over Time for Reward Configuration {vals} with Max-Count {'On' if max_count else 'Off'}""", log_y=log_y)

In [34]:
plot_vals((1, 2, 3, 4, 5), max_count=True, i=2)

In [35]:
plot_vals((1, 2, 3, 4, 5), max_count=False, i=2)

In [36]:
plot_vals((4, 2, 3, 1, 5), max_count=True, log_y=True, i=0)

In [37]:
plot_vals((4, 2, 3, 1, 5), max_count=False, log_y=True, i=1)